In [8]:
import os
import sys
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

sys.path.append("python")
from utils import print_parameters

In [9]:
model_architecture = ["resnet", "swin"][0]
seed = 19
# checkpoint_path = "../model_weights/PDDD/ResNet50-Plant-model-80.pth"
# checkpoint_path = "../model_weights/PDDD/swin_tiny_patch4_window7_224-model-97.pth"
checkpoint_path = None
num_classes = None
pretrained = True
# num_classes = 120

# weights_path = f"../model_weights/random/ResNet50-Random_{seed}.pth"
# weights = None

weights_path = "../model_weights/imagenet/ResNet50-ImageNet_v1.pth"
# weights_path = "../model_weights/imagenet/ResNet50-ImageNet_v2.pth"
weights = ResNet50_Weights.IMAGENET1K_V1
# weights = ResNet50_Weights.IMAGENET1K_V2

# weights_path = f"../model_weights/random/Swin_T-Random_{seed}.pth"
# weights_path = "../model_weights/imagenet/Swin_T-ImageNet.pth"

assert model_architecture in weights_path.lower()

In [10]:
torch.manual_seed(seed)

In [11]:
model = None

if model_architecture == "resnet":
    model = resnet50(weights=None)
    print(f"Output size: {model.fc.out_features}")
    if num_classes is not None:
        model.fc = nn.Linear(model.fc.in_features, num_classes)

elif model_architecture == "swin":
    print(f"pretrained: {pretrained}")
    model = torch.hub.load(
        "SharanSMenon/swin-transformer-hub:main",
        "swin_tiny_patch4_window7_224",
        pretrained=pretrained,
    )
    print(f"Output size: {model.head.out_features}")
    if num_classes is not None:
        model.head = nn.Linear(model.head.in_features, num_classes)

else:
    print(f"Unknown model architecture: {model_architecture}")

print_parameters(model)

Output size: 1000
Total parameters: 25557032
Required parameters: 25557032 


(25557032, 25557032)

In [12]:
if checkpoint_path is not None:
    checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint, strict=False)

In [14]:
print_parameters(model)  # 25557032 / 28288354
if os.path.exists(weights_path):
    print(f"File {weights_path} already exists")
else:
    torch.save(model.state_dict(), weights_path)
    print(f"File {weights_path} saved")

Total parameters: 25557032
Required parameters: 25557032 
File ../model_weights/imagenet/ResNet50-ImageNet_v1.pth saved
